In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import datetime
import json
from pandas.io.json import json_normalize
import csv

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = 999
pd.set_option('display.max_row', 1000)

In [ ]:
pin = pd.read_csv('API Reports/report_dataset1.tsv',sep = '\t')

cel_cdl  = pd.read_csv('API Reports/report_dataset2.tsv',sep = '\t')

rw = pd.read_csv('API Reports/report_dataset3.tsv',sep = '\t')

zp = pd.read_csv('API Reports/report_dataset4.tsv',sep = '\t')

px = pd.read_csv('API Reports/report_dataset5.tsv',sep = '\t',lineterminator='\n')

zp['Date'] = pd.to_datetime(zp['Date'])
px['Local Date'] = pd.to_datetime(px['Local Date'])
pin['Date'] = pd.to_datetime(pin['Date'])


zp = zp[~((zp['Polygon ID']=="McDonald's 6676, Chicago, IL | 98504") & (zp['Date'] < '2018-08-01'))]
px = px[~((px['Study Polygon'] == "McDonald's 6676, Chicago, IL | 98504")&(px['Local Date'] < '2018-08-01'))]
pin = pin[~((pin['Polygon ID'] == "McDonald's 6676, Chicago, IL | 98504")&(pin['Date'] < '2018-08-01'))]

filtered_zp = zp[~(zp['UDID'].isin(rw['Device ID']))]
filtered_px = px[~(px['UDID'].isin(rw['Device ID']))]
filtered_pin = pin[~(pin['UDID'].isin(rw['Device ID']))]
filtered_cel_cdl = cel_cdl[~(cel_cdl['UDID'].isin(rw['Device ID']))]


filtered_zp['Time of Day'] = pd.to_timedelta(filtered_zp['Time of Day'])
filtered_px['Local Time of Day'] = pd.to_timedelta(filtered_px['Local Time of Day'])

In [3]:
starting_devices = filtered_zp.groupby(['UDID','Date']).count().reset_index()

unknowns = starting_devices[starting_devices['Unix Timestamp of Visit'] < 2]
dine_in = starting_devices[starting_devices['Unix Timestamp of Visit'] >= 2]

In [4]:
starting_devices_list = starting_devices['UDID'].drop_duplicates().tolist()

In [5]:
filtered_zp['Polygon ID'].value_counts()

Portillos,  Chicago,  IL | 7962939                            179610
McDonald's 6676, Chicago, IL | 98504                          161991
Maggiano's Little Italy 207, Chicago, IL | 4392516             52033
Tempo Cafe,  Chicago,  IL | 7962942                            48508
TGI Fridays 2668 - Chicago - Erie St, Chicago, IL | 390511     40330
Lou Malnatis,  Chicago,  IL | 7962940                          26800
Velvet Taco,  Chicago,  IL | 7962941                           25686
Seoul Taco,  Chicago,  IL | 7962943                            20561
Hooters 231 - Chicago - Wells St., Chicago, IL | 3519707       15294
The Halal Guys,  Chicago,  IL | 7962944                         8424
Name: Polygon ID, dtype: int64

In [6]:
filtered_zp['UDID'].nunique()
dine_in['UDID'].nunique()
unknowns['UDID'].nunique()

59817

40836

26095

# Device Sets:



* 59,817 devices in total
* 40,846 devices to start with
* 26,095 devices to determine if they were seen in dine in areas. Devices that arent filtered belong in unknown category


In [7]:
customer_devices = dine_in['UDID'].drop_duplicates().tolist()

In [8]:
# Filter reports further with customer_device list

filtered_zp = zp[zp['UDID'].isin(customer_devices)]
filtered_px = px[px['UDID'].isin(customer_devices)]
filtered_pin = pin[pin['UDID'].isin(customer_devices)]
filtered_cel_cdl = cel_cdl[cel_cdl['UDID'].isin(customer_devices)]

In [9]:
filtered_cel_cdl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55830 entries, 1 to 82948
Data columns (total 16 columns):
Polygon Name               55830 non-null object
Unhashed Device Id         55830 non-null object
Common Evening Lat         53754 non-null float64
Common Evening Long        53754 non-null float64
Common Evening Country     53718 non-null object
Common Evening State       52369 non-null object
Common Evening Zip         52735 non-null object
Common Evening Census      52919 non-null object
Common Evening Distance    53754 non-null float64
Common Daytime Lat         50857 non-null float64
Common Daytime Long        50857 non-null float64
Common Daytime Country     50818 non-null object
Common Daytime State       49578 non-null object
Common Daytime Zip         49924 non-null object
Common Daytime Census      50097 non-null object
Common Daytime Distance    50857 non-null float64
dtypes: float64(6), object(10)
memory usage: 7.2+ MB


In [10]:
zero_1st = filtered_px.drop_duplicates(subset = ['UDID','Local Date',
                                                 'Time before appearance in polygon'], keep = 'first')

zero_last = filtered_px.drop_duplicates(subset = ['UDID','Local Date',
                                                 'Time before appearance in polygon'], keep = 'last')


In [11]:
zero_1st = zero_1st[zero_1st['Time before appearance in polygon'] == 0]
zero_last = zero_last[zero_last['Time before appearance in polygon'] == 0]

In [12]:
zero_1st.drop(['Lat of Observation Point','Lon of Observation Point','Unix Timestamp of Observation Point',
             'Local Day of Week','Local Timezone of Observation Point','Polygon Id',
             'Polygon Name','Category'], axis = 1, inplace = True)

zero_last.drop(['Lat of Observation Point','Lon of Observation Point','Unix Timestamp of Observation Point',
             'Local Day of Week','Local Timezone of Observation Point','Polygon Id',
             'Polygon Name','Category'], axis = 1, inplace = True)

In [19]:
dwell_time = zero_1st.merge(zero_last,how = 'inner',on = ['Study Polygon','UDID','Local Date'])

In [20]:
dwell_time.drop(['Time before appearance in polygon_x','Time before appearance in polygon_y'],
               axis = 1, inplace = True)
dwell_time.rename(columns= {'Local Time of Day_x':'Initial PIP Time','Local Time of Day_y':'Final PIP Time'},
                 inplace = True)

In [21]:
dwell_time['Initial PIP Time'] = pd.to_timedelta(dwell_time['Initial PIP Time'])
dwell_time['Final PIP Time'] = pd.to_timedelta(dwell_time['Final PIP Time'])

In [22]:
dwell_time['Time in Point'] = dwell_time['Final PIP Time'] - dwell_time['Initial PIP Time']

In [23]:
TO_DD_dwell = dwell_time[(dwell_time['Time in Point'] <= '00:07:00') & (dwell_time['Time in Point'] > '00:00:30')]

In [24]:
dine_in_dwell = dwell_time[(dwell_time['Time in Point'] > '00:07:00') & (dwell_time['Time in Point'] <= '02:00:00')]
double_visits = dwell_time[dwell_time['Time in Point'] > '02:00:00']

In [25]:
unknown_dwell = dwell_time[dwell_time['Time in Point'] <= '00:00:30']

In [26]:
TO_DD_dwell = TO_DD_dwell[TO_DD_dwell['UDID'].isin(starting_devices_list)]
dine_in_dwell = dine_in_dwell[dine_in_dwell['UDID'].isin(starting_devices_list)]
double_visits = double_visits[double_visits['UDID'].isin(starting_devices_list)]
unknown_dwell = unknown_dwell[unknown_dwell['UDID'].isin(starting_devices_list)]

In [27]:
TO_DD_dwell['UDID'].nunique()

8444

In [28]:
unknown_dwell['UDID'].nunique()

16773

In [29]:
dine_in_dwell['UDID'].nunique()

11201

In [30]:
double_visits['UDID'].nunique()

13004

According to this filtering, here are the device breakdowns:
* To-go/Drivers: 4,783
* Unknown Dwell: 11,569
* Dine in Dwell: 5,530
* Double Visits: 10,204


Do some further filtering on the double visits!

# Merge CEL/CDL to Pathing X

In [31]:
filtered_cel_cdl.columns

Index(['Polygon Name', 'Unhashed Device Id', 'Common Evening Lat',
       'Common Evening Long', 'Common Evening Country', 'Common Evening State',
       'Common Evening Zip', 'Common Evening Census',
       'Common Evening Distance', 'Common Daytime Lat', 'Common Daytime Long',
       'Common Daytime Country', 'Common Daytime State', 'Common Daytime Zip',
       'Common Daytime Census', 'Common Daytime Distance'],
      dtype='object')

In [ ]:
cels_cdls = filtered_cel_cdl[['UDID', 'Common Evening Lat',
       'Common Evening Long','Common Daytime Lat', 'Common Daytime Long',]]
cels_cdls.rename(columns = {'UDID':'UDID'}, inplace = True)

In [33]:
updated_px = filtered_px.merge(cels_cdls,how = 'left', on = ['UDID'])

In [34]:
updated_px = updated_px[(updated_px['Common Evening Lat'].isnull() == False) & 
                        (updated_px['Common Daytime Lat'].isnull() == False)]

updated_px.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8825760 entries, 0 to 9654971
Data columns (total 17 columns):
Study Polygon                          object
Unhashed Device ID                     object
Lat of Observation Point               float64
Lon of Observation Point               float64
Time before appearance in polygon      int64
Unix Timestamp of Observation Point    int64
Local Date                             datetime64[ns]
Local Time of Day                      object
Local Day of Week                      object
Local Timezone of Observation Point    object
Polygon Id                             float64
Polygon Name                           object
Category                               object
Common Evening Lat                     float64
Common Evening Long                    float64
Common Daytime Lat                     float64
Common Daytime Long                    float64
dtypes: datetime64[ns](1), float64(7), int64(2), object(7)
memory usage: 1.2+ GB


In [35]:
updated_px.columns

Index(['Study Polygon', 'Unhashed Device ID', 'Lat of Observation Point',
       'Lon of Observation Point', 'Time before appearance in polygon',
       'Unix Timestamp of Observation Point', 'Local Date',
       'Local Time of Day', 'Local Day of Week',
       'Local Timezone of Observation Point', 'Polygon Id', 'Polygon Name',
       'Category', 'Common Evening Lat', 'Common Evening Long',
       'Common Daytime Lat', 'Common Daytime Long'],
      dtype='object')

In [36]:
updated_px = pd.read_csv('final.tsv', sep = '\t',header =None, names =['Study Polygon', 'UDID', 'Lat of Observation Point',
       'Lon of Observation Point', 'Time before appearance in polygon',
       'Unix Timestamp of Observation Point', 'Local Date',
       'Local Time of Day', 'Local Day of Week',
       'Local Timezone of Observation Point', 'Polygon Id', 'Polygon Name',
       'Category', 'Common Evening Lat', 'Common Evening Long',
       'Common Daytime Lat', 'Common Daytime Long', 'CEL Distance', 'CDL Distance'],lineterminator='\n' )

In [37]:
togocelcdl_px = updated_px[(updated_px['CEL Distance'] < 20) |(updated_px['CDL Distance'] < 20) ]

In [38]:
togocelcdl_px = togocelcdl_px[togocelcdl_px['Time before appearance in polygon'] < -60]

In [39]:
togocelcdl_px[togocelcdl_px['UDID'].isin(TO_DD_dwell['UDID'])]['UDID'].nunique()

837

* Out of the 9780 devices that had a dwell time of less than 7 mins, only 672 ended at their CEL/CDL
* 4227 devices started and ended from their CEL/CDL

In [40]:
#All Devices from starting pool
togocelcdl_px['UDID'].nunique()

4227

using a 20 meter buffer zone from CEL/CDL, found that 6108 devices showed togo device habits

In [41]:
filtered_cel_cdl.columns

Index(['Polygon Name', 'Unhashed Device Id', 'Common Evening Lat',
       'Common Evening Long', 'Common Evening Country', 'Common Evening State',
       'Common Evening Zip', 'Common Evening Census',
       'Common Evening Distance', 'Common Daytime Lat', 'Common Daytime Long',
       'Common Daytime Country', 'Common Daytime State', 'Common Daytime Zip',
       'Common Daytime Census', 'Common Daytime Distance'],
      dtype='object')

In [ ]:
non_chi_cels = pd.read_csv('non-Chicago_cels.tsv', sep = '\t', header = None, names = ['Polygon Name', 'UDID', 'Common Evening Lat',
       'Common Evening Long', 'Common Evening Country', 'Common Evening State',
       'Common Evening Zip', 'Common Evening Census',
       'Common Evening Distance', 'Common Daytime Lat', 'Common Daytime Long',
       'Common Daytime Country', 'Common Daytime State', 'Common Daytime Zip',
       'Common Daytime Census', 'Common Daytime Distance'])
non_chi_cels

In [43]:
non_chi_cels.dropna(subset=['Common Evening Lat'], inplace = True)

In [44]:
non_chi_cels['UDID'].nunique()

26672

In [45]:
non_chi_2go = TO_DD_dwell[TO_DD_dwell['UDID'].isin(non_chi_cels['UDID'])]
TO_DD_dwell[TO_DD_dwell['UDID'].isin(non_chi_cels['UDID'])]['UDID'].nunique()

2793

70 devices from the pool of 4227 to go devices based on their CEL have CELS outside of the Chicago Metro area!

# To Summarize:
 ## 44,895 starting devices
 ## 12,048 dine in based on dwell
 ## 9780 To-go/Driver devices based on dwell
 ## Out of 9780 devices, only 953 devices ended at CEL/CDL
 ## 4227 devices ended at their CEL/CDL
 ## 70 devices had CELs outside of Chicago metro area

# Number of Visits in a Year

In [ ]:
# Driver devices are those with more than 200 visits in a year
driver = filtered_zp[filtered_zp.groupby("Unhashed Device ID")["Unhashed Device ID"].transform('size') >= 200].reset_index(drop=True)

driver = driver[driver['UDID'].isin(TO_DD_dwell['UDID'])]
driver = driver[~driver['UDID'].isin(togocelcdl_px['UDID'])]
driver['UDID'].nunique()
driver_dids = driver[['Polygon ID','UDID']].drop_duplicates()
driver_dids['UDID']
driver_dids['Polygon ID'].value_counts().reset_index()


This logic returns 36 driver devices

In [53]:
filtered_zp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558336 entries, 0 to 668968
Data columns (total 7 columns):
Polygon ID                 558336 non-null object
Unhashed Device ID         558336 non-null object
Unix Timestamp of Visit    558336 non-null int64
Date                       558336 non-null datetime64[ns]
Time of Day                558336 non-null object
Day of Week                558336 non-null object
Time Zone                  558336 non-null object
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 34.1+ MB


In [54]:
filtered_zp['Time of Day'] = pd.to_timedelta(filtered_zp['Time of Day'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Create time delta col to determine seperation of visits greater than 1 hr
filtered_zp['time_delta'] = (filtered_zp['Time of Day'] - filtered_zp['Time of Day'].shift())
daily_visits = filtered_zp[(filtered_zp['time_delta']>'01:00:00') & (filtered_zp['UDID'].duplicated() == True) & 
           (filtered_zp['Date'].duplicated() ==True)]

In [56]:
# Countthe number of times a device repeated a visit in a day. Filter out devices with more than 2 visits
daily_visit_count = daily_visits.groupby(['UDID','Date']).count().reset_index()
possible_drivers = daily_visit_count[daily_visit_count['Time of Day'] > 2]

In [57]:
possible_drivers = possible_drivers[possible_drivers['UDID'].isin(TO_DD_dwell['UDID'])]

In [58]:
driver_dids_daily = possible_drivers[['UDID']].drop_duplicates()
#driver_dids_daily = driver_dids_daily['UDID'].tolist()

In [59]:
driver_dids_daily = driver_dids_daily[~driver_dids_daily['UDID'].isin(togocelcdl_px['UDID'])]

In [60]:
#driver_dids_daily = driver_dids_daily['UDID'].tolist()

In [61]:
len(driver_dids_daily)

214

This logic returns 142 devices!

In [62]:
filtered_driver_dids = driver_dids_daily[driver_dids_daily['UDID'].isin(driver_dids['UDID']) == True]

In [63]:
filtered_driver_dids = filtered_driver_dids['UDID']

In [64]:
filtered_driver_dids.nunique()

41

* 25 devices have both of these traits, where they have more than 200 visits a year and have more than 2 visits in a day!

# Going back to unknown dwell devices and double visits

In [65]:
unknown_dwell['UDID'].nunique()
double_visits['UDID'].nunique()

16773

13004

In [66]:
# CELCDL ending location in unknown dwell
togocelcdl_px[togocelcdl_px['UDID'].isin(unknown_dwell['UDID'])]['UDID'].nunique()

2085

In [67]:
# CELCDL ending location in double visits
togocelcdl_px[togocelcdl_px['UDID'].isin(double_visits['UDID'])]['UDID'].nunique()

2018

In [68]:
# More than 200 visits annually devices for unknown dwell devices
driver[driver['UDID'].isin(unknown_dwell['UDID'])]['UDID'].nunique()
# More than 200 visits annually devices for double visit dwell devices
driver[driver['UDID'].isin(double_visits['UDID'])]['UDID'].nunique()

47

49

In [69]:
# Unknown dwell devices with more than 2 visits in a day
daily_visit_count[(daily_visit_count['UDID'].isin(unknown_dwell['UDID']))&
                 (daily_visit_count['Time of Day'] > 2)]['UDID'].nunique()
# Double visit devices with more than 2 visits in a day
daily_visit_count[(daily_visit_count['UDID'].isin(double_visits['UDID']))&
                 (daily_visit_count['Time of Day'] > 2)]['UDID'].nunique()

684

1053

In [70]:
# Unknown dwell devices with more than 2 visits in a day
daily_visit_count[(daily_visit_count['UDID'].isin(unknown_dwell['UDID']))&
                 (daily_visit_count['Time of Day'] > 2) &
                 (daily_visit_count['UDID'].isin(driver['UDID']))]['UDID'].nunique()
# Double visit devices with more than 2 visits in a day
daily_visit_count[(daily_visit_count['UDID'].isin(double_visits['UDID']))&
                 (daily_visit_count['Time of Day'] > 2) &
                 (daily_visit_count['UDID'].isin(driver['UDID']))]['UDID'].nunique()

38

39

## Summary
### Out of 11569 devices with unknown dwell time:
    * 1,796 devices ended at CEL/CDL
    * 27 devices visited a location more than 200 times in the year
    * 539 devices had more than 2 visits in a day
    * 21 of devices with more than 2 visits + 200 visits
### Out of 10204 devices with double visits dwell time:
    * 1,758 devices started/ended from CEL/CDL
    * 29 devices visited a location more than 200 times in the year
    * 821 devices had more than 2 visits in a day
    * 22 of devices with more than 2 visits + 200 visits
    
Adding these 1796 + 1758 devices to potential togo customer pool

Adding these 21 + 22 devices to potential driver pool

In [71]:

a = daily_visit_count[(daily_visit_count['UDID'].isin(unknown_dwell['UDID']))&
                 (daily_visit_count['Time of Day'] > 2) &
                 (daily_visit_count['UDID'].isin(driver['UDID']))]['UDID']

b = daily_visit_count[(daily_visit_count['UDID'].isin(double_visits['UDID']))&
                 (daily_visit_count['Time of Day'] > 2) &
                 (daily_visit_count['UDID'].isin(driver['UDID']))]['UDID']

driver_pool_unknowns = pd.concat([a,b])
driver_pool_unknowns.drop_duplicates(inplace = True)
driver_pool_unknowns.shape

(41,)

In [72]:
a = togocelcdl_px[togocelcdl_px['UDID'].isin(unknown_dwell['UDID'])]['UDID']

b = togocelcdl_px[togocelcdl_px['UDID'].isin(double_visits['UDID'])]['UDID']
to_go_pool_unknowns = pd.concat([a,b])
to_go_pool_unknowns.drop_duplicates(inplace = True)
to_go_pool_unknowns.shape

(2966,)

# Adding these devices together, a net increase of togos = 2612 and drivers = 25

In [73]:
beginnning_unknowns_dids = unknowns['UDID'].drop_duplicates().tolist()

In [74]:
len(beginnning_unknowns_dids)

26095

In [75]:
unknown_dwell_dids = unknown_dwell['UDID'].drop_duplicates().tolist()

In [76]:
len(unknown_dwell_dids)

16773

In [77]:
total_unknowns = unknown_dwell_dids + beginnning_unknowns_dids

In [78]:
total_unknowns = list(set(total_unknowns))

In [79]:
len(total_unknowns)

36303

## 32,914 unique devices that are unknowns

In [80]:
filtered_px[filtered_px['UDID'].isin(total_unknowns)].to_csv('uknown_devices_pathx.tsv', sep = '\t',
                                                                       index = False)

In [81]:
filtered_px.columns

Index(['Study Polygon', 'Unhashed Device ID', 'Lat of Observation Point',
       'Lon of Observation Point', 'Time before appearance in polygon',
       'Unix Timestamp of Observation Point', 'Local Date',
       'Local Time of Day', 'Local Day of Week',
       'Local Timezone of Observation Point', 'Polygon Id', 'Polygon Name',
       'Category'],
      dtype='object')

# Unknown Devices PathX filtered with possible dine in areas of restaurants

In [82]:
u_pathx = pd.read_csv('uknown_devices_pathx_dineins.tsv',sep = '\t', names = ['Study Polygon', 'UDID', 'Lat of Observation Point',
       'Lon of Observation Point', 'Time before appearance in polygon',
       'Unix Timestamp of Observation Point', 'Local Date',
       'Local Time of Day', 'Local Day of Week',
       'Local Timezone of Observation Point', 'Polygon Id', 'Polygon Name',
       'Category','PNP Location'])

In [83]:
u_pathx = u_pathx[~((u_pathx['Study Polygon'] != "POI")&(u_pathx['Local Date'] < '2018-08-01'))]

In [84]:
u_pathx['UDID'].nunique()

8597

## Potential Dine in Devices added: 8,597 from the unknown devices due to the location the point was seen

In [85]:
din_unknown_dwell_dids = u_pathx['UDID'].drop_duplicates().tolist()
din_devices = dine_in_dwell['UDID'].drop_duplicates().tolist()

In [86]:
len(din_devices)

11201

In [87]:
total_dineins = din_devices + din_unknown_dwell_dids
total_dineins = list(set(total_dineins))
len(total_dineins)

18066

# Total Dine In Devices = 12,688

In [88]:
dd_1 = driver_dids['UDID'].drop_duplicates().tolist()
dd_2 = driver_dids_daily['UDID'].drop_duplicates().tolist()
driver_pool = dd_1 + dd_2 + driver_pool_unknowns.tolist()
driver_pool = list(set(driver_pool))
len(driver_pool)

228

# Total Driver Devices = 150

In [89]:
togo_dev_pool = togocelcdl_px[togocelcdl_px['UDID'].isin(TO_DD_dwell['UDID'])]['UDID'].tolist()

In [90]:
togo_pool = togo_dev_pool + to_go_pool_unknowns.tolist()
togo_pool = list(set(togo_pool))
len(togo_pool)

3136

# Total ToGo Devices = 2,738

# Out of 66,320 Total starting devices

In [91]:
len(set(total_dineins).intersection(togo_pool))

1728

# Comparing Dine Ins to ToGo, 1,579 of dine ins were also to go's

In [92]:
len(set(total_dineins).intersection(driver_pool))

166

# Comparing Dine Ins to Drivers, 112 of dine ins were also drivers

In [93]:
len(set(togo_pool).intersection(driver_pool))

0

# Comparing ToGo to Drivers,  0 of togo were also drivers

In [94]:
len(set(total_unknowns).intersection(driver_pool))
len(set(total_unknowns).intersection(togo_pool))
len(set(total_unknowns).intersection(total_dineins))

170

2101

8725

In [95]:
zero_drivers = filtered_zp[filtered_zp['UDID'].isin(driver_pool)]
zero_drivers = zero_drivers[['Polygon ID','UDID']].drop_duplicates()
zero_drivers['Polygon ID'].value_counts().reset_index()

,index,Polygon ID
0,"Portillos, Chicago, IL | 7962939",119
1,"McDonald's 6676, Chicago, IL | 98504",103
2,"Maggiano's Little Italy 207, Chicago, IL | 439...",46
3,"Tempo Cafe, Chicago, IL | 7962942",44
4,"TGI Fridays 2668 - Chicago - Erie St, Chicago,...",35
5,"Seoul Taco, Chicago, IL | 7962943",28
6,"Hooters 231 - Chicago - Wells St., Chicago, IL...",25
7,"Velvet Taco, Chicago, IL | 7962941",23
8,"Lou Malnatis, Chicago, IL | 7962940",19
9,"The Halal Guys, Chicago, IL | 7962944",7


In [96]:
zero_togo = filtered_zp[filtered_zp['UDID'].isin(togo_pool)]
zero_togo = zero_togo[['Polygon ID','UDID']].drop_duplicates()
zero_togo['Polygon ID'].value_counts().reset_index()

,index,Polygon ID
0,"McDonald's 6676, Chicago, IL | 98504",1549
1,"Portillos, Chicago, IL | 7962939",1417
2,"Maggiano's Little Italy 207, Chicago, IL | 439...",609
3,"Tempo Cafe, Chicago, IL | 7962942",561
4,"Velvet Taco, Chicago, IL | 7962941",442
5,"Hooters 231 - Chicago - Wells St., Chicago, IL...",329
6,"Seoul Taco, Chicago, IL | 7962943",319
7,"TGI Fridays 2668 - Chicago - Erie St, Chicago,...",306
8,"Lou Malnatis, Chicago, IL | 7962940",233
9,"The Halal Guys, Chicago, IL | 7962944",216


In [97]:
zero_dinein = filtered_zp[filtered_zp['UDID'].isin(total_dineins)]
zero_dinein = zero_dinein[['Polygon ID','UDID']].drop_duplicates()
zero_dinein['Polygon ID'].value_counts().reset_index()

,index,Polygon ID
0,"Portillos, Chicago, IL | 7962939",8538
1,"McDonald's 6676, Chicago, IL | 98504",5512
2,"Tempo Cafe, Chicago, IL | 7962942",2290
3,"Maggiano's Little Italy 207, Chicago, IL | 439...",2167
4,"Velvet Taco, Chicago, IL | 7962941",1440
5,"TGI Fridays 2668 - Chicago - Erie St, Chicago,...",1433
6,"Lou Malnatis, Chicago, IL | 7962940",1351
7,"Hooters 231 - Chicago - Wells St., Chicago, IL...",991
8,"Seoul Taco, Chicago, IL | 7962943",976
9,"The Halal Guys, Chicago, IL | 7962944",542


In [98]:
dine_test = filtered_px[(filtered_px['UDID'].isin(din_devices))]

In [99]:
dine_testpx = dine_test[~((dine_test['Study Polygon'] == "McDonald's 6676, Chicago, IL | 98504")&(dine_test['Local Date'] < '2018-08-01'))]

In [101]:
take_test = filtered_px[(filtered_px['UDID'].isin(togo_pool))]
take_test = take_test[~((take_test['Study Polygon'] == "McDonald's 6676, Chicago, IL | 98504")&
                        (take_test['Local Date'] < '2018-08-01'))]

In [103]:
driver_test = filtered_px[(filtered_px['UDID'].isin(driver_pool))]
driver_test = driver_test[~((driver_test['Study Polygon'] == "McDonald's 6676, Chicago, IL | 98504")&
                        (driver_test['Local Date'] < '2018-08-01'))]